In [1]:
import os

import espnet2.gan_tts.hifigan

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from pathlib import Path
from espnet2.text.phoneme_tokenizer import PhonemeTokenizer
from espnet2.text.token_id_converter import TokenIDConverter
from espnet2.bin.tts_inference import Text2Speech

from prosody.pinyin import *
from prosody.en_to_zh import PinyinArpaSpeech, all_tones

PWD = %pwd
PWD = Path(PWD)
LJSPEECH_DIR = (PWD / '../egs2/ljspeech/tts1/').resolve()
device = 'cuda'

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
LJSPEECH_DIR

PosixPath('/home/perry/PycharmProjects/present/egs2/ljspeech/tts1')

In [3]:
os.chdir(LJSPEECH_DIR)
pretrained_dir = LJSPEECH_DIR / "exp/tts_train_jets_raw_phn_tacotron_g2p_en_no_space"
pretrained_model_file = pretrained_dir / "train.total_count.ave_5best.pth"
pretrained_tts = Text2Speech.from_pretrained(
    train_config=pretrained_dir / "config.yaml",
    model_file=pretrained_model_file,
    device=device
)
pretrained_model = pretrained_tts.model
os.chdir(PWD)

AttributeError: 'HiFiGANScaleDiscriminator' object has no attribute 'register_load_state_dict_post_hook'

In [1]:
%load_ext autoreload
%autoreload 1
%aimport espnet2.gan_tts.hifigan
from espnet2.gan_tts.hifigan import HiFiGANScaleDiscriminator
sd = HiFiGANScaleDiscriminator()

In [4]:
import torch
for x in dir(torch.nn.Module):
    print(x)

T_destination
__annotations__
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattr__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_apply
_call_impl
_get_backward_hooks
_get_name
_load_from_state_dict
_maybe_warn_non_full_backward_hook
_named_members
_register_load_state_dict_pre_hook
_register_state_dict_hook
_replicate_for_data_parallel
_save_to_state_dict
_slow_forward
_version
add_module
apply
bfloat16
buffers
children
cpu
cuda
double
dump_patches
eval
extra_repr
float
forward
get_buffer
get_extra_state
get_parameter
get_submodule
half
load_state_dict
modules
named_buffers
named_children
named_modules
named_parameters
parameters
register_backward_hook
register_buffer
register_forward_hook
register_forward_pre_hook
register_full_backward_hook
register_module
register_parameter
requires_g

In [3]:

for x in dir(sd):
    print(x)

T_destination
__annotations__
__call__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattr__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_apply
_backward_hooks
_buffers
_call_impl
_forward_hooks
_forward_pre_hooks
_get_backward_hooks
_get_name
_is_full_backward_hook
_load_from_state_dict
_load_state_dict_post_hook
_load_state_dict_pre_hook
_load_state_dict_pre_hooks
_maybe_warn_non_full_backward_hook
_modules
_named_members
_non_persistent_buffers_set
_parameters
_register_load_state_dict_pre_hook
_register_state_dict_hook
_replicate_for_data_parallel
_save_to_state_dict
_slow_forward
_state_dict_hooks
_version
add_module
apply
apply_spectral_norm
apply_weight_norm
bfloat16
buffers
children
cpu
cuda
double
dump_patches
eval
extra_repr
float
forward
get_buffer
get_extra_state
get_parameter
get_sub

In [ ]:
arpa_tokenizer = PhonemeTokenizer(g2p_type='g2p_en_no_space')
id_converter = TokenIDConverter(pretrained_tts.train_args.token_list)

In [ ]:
save_dir = PWD / 'tts_train_jets_raw_phn_tacotron_g2p_en_no_space/mandarin'
pas = PinyinArpaSpeech(token_id_converter=id_converter, tts_inference_fn=pretrained_model.tts.generator.inference)

In [ ]:
initial_pinyins = INITIAL_TO_PINYINS['zh']
initial_pinyin_tones = all_tones(initial_pinyins)
' '.join(initial_pinyin_tones)

In [ ]:
rime_pinyins = RIME_TO_PINYINS['uang']
rime_pinyin_tones = all_tones(rime_pinyins)
' '.join(rime_pinyin_tones)

In [ ]:
chinese = all_tones('zhuang') + ['.']
# chinese = rime_pinyin_tones[0:20] + ['.']
# chinese = rime_pinyin_tones[20:40] + ['.']
# chinese = rime_pinyin_tones[40:60] + ['.']
# chinese = rime_pinyin_tones[60:80] + ['.']
# chinese = '我提到的这个问题并不难处理.'
# chinese = '这个议会代表着欧洲民众.'
# chinese = '我相信你在那裡涉及到了某個要點.'
# chinese = '各項報導反映出這種主流態度.'
# chinese = initial_pinyin_tones[0:20] + ['.']
# chinese = initial_pinyin_tones[20:40] + ['.']
# chinese = initial_pinyin_tones[40:60] + ['.']
# chinese = '他晕倒了.'

pas_update_dict = {
    'pinyin_to_arpa_durations': {},
    'tone_duration_split': {},
    'tone_contours': {},
    'nucleus_tone_only': False,
    'max_pitch_change': 2.5,
}
infer_overrides = {
    # 'd_split_factor': None,
    # 'd_factor': None,
    # 'p_factor': None,
    # 'e_factor': None,
    # 'd_mod_fns': None,
    # 'p_mod_fns': None,
    # 'e_mod_fns': None,
}
inputs = pas.gen_audio(
    chinese,
    save_dir,
    inputs=None,
    pac_update_dict=pas_update_dict,
    infer_overrides=infer_overrides,
    overall_d_factor=1.0,
    vowel_duration=(9.0, 15.0),
    arpa_in_filename=len(chinese)<10,
    device=device,
)
arpas, tones, d_factor, d_split_factor, pitch_values, p_mod_fns = inputs